In [66]:
'''
This Notebook uses cloudFoundary services from GE Predix
to demonstrate the use of authentication and asset management
in the context of a simple user application.

We use:
    subprocess management (Python standard library)
    json (Python standard library)
    cloudFoundry api (via linux "curl")
    cloudFoundry cf CLI
    cloufFoundary UAA and Asset micrososervices

The script deploys UAA and Asset microservices and uses a
a simple user web app to demonstrate authentication.

References:
The process is based on the example at
    https://www.predix.io/blog/article.html?article_id=2034
with additional information from 
    http://stackoverflow.com/questions/27985469/how-to-get-oauth-token-from-cloudfoundry
    https://docs.python.org/3.7/reference/index.html
    https://docs.python.org/3.7/library/index.html
'''
x=1

In [67]:
### Log In

import getpass
from subprocess import check_output

print("cloudForge log in")
u = raw_input("username:")
p = getpass.getpass("password:")
foo = check_output(["cf", "login", "-u", u, "-p", p])
#print foo

print("done")

cloudForge log in
username:cwinsor@gmail.com
password:········
done


In [68]:
# Identify what space to target.  Get the URL

from subprocess import check_output
import json

foo = check_output(["cf", "curl", "/v2/spaces"])
MY_PR_ORG = json.loads(foo)
#print json.dumps(MY_PR_ORG, sort_keys=True, indent=4, separators=(',', ': '))

dict = {}
for num, line in enumerate(MY_PR_ORG["resources"]):
    dict[num] = line["entity"]["name"]

if (len(dict)==1):
    sp = 0;
else:
    finished = False
    while not finished:
        sp = input("Choose the space you want to target:\n     " + str(dict))
        try:
            t = MY_PR_ORG["resources"][sp]
            finished = True
        except IndexError:
            finished = False

MY_PR_SPACE_URL    = MY_PR_ORG["resources"][sp]["metadata"]["url"]
MY_PR_APPS_URL     = MY_PR_ORG["resources"][sp]["entity"]["apps_url"]
MY_PR_SERVICES_URL = MY_PR_ORG["resources"][sp]["entity"]["service_instances_url"]

print "MY_PR_SPACE_URL " + MY_PR_SPACE_URL
print "MY_PR_APPS_URL " +  MY_PR_APPS_URL
print "MY_PR_SERVICES_URL " +  MY_PR_SERVICES_URL

root = json.loads(check_output(["cf", "curl", MY_PR_SPACE_URL]))
print "name: " + root["entity"]["name"]
print json.dumps(root, sort_keys=True, indent=4, separators=(',', ': '))

# tell cf to target that space
foo = check_output(["cf", "target", "-s", MY_PR_SPACE_NAME])

Choose the space you want to target:
     {0: u'dev', 1: u'other'}0
MY_PR_SPACE_URL /v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc
MY_PR_APPS_URL /v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/apps
MY_PR_SERVICES_URL /v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/service_instances
name: dev
{
    "entity": {
        "allow_ssh": true,
        "app_events_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/app_events",
        "apps_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/apps",
        "auditors_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/auditors",
        "developers_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/developers",
        "domains_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/domains",
        "events_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/events",
        "isolation_segment_guid": null,
        "managers_url": "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/managers",
        "name": "dev",
        "organi

In [69]:
### Create UAA Service
### cf create-service predix-uaa Free my-uaa -c '{"adminClientSecret":"my-secret"}'

from subprocess import check_output
import json

print("Create UAA Service")
secret = getpass.getpass('secret code:')
aa = "\"adminClientSecret\""
bb = "\"" + secret + "\""
cc = "{" + aa + ":" + bb + "}"
#print cc
dd = ["cf", "create-service", "predix-uaa", "Free", "my-uaa", "-c", cc]
#print dd
foo = check_output(dd)
print foo

# now get the URL for the UAA
root = json.loads(check_output(["cf", "curl", MY_PR_SERVICES_URL]))
#print json.dumps(root, sort_keys=True, indent=4, separators=(',', ': '))
# look for "my-uaa" service
for num, line in enumerate(root["resources"]):
    if (line["entity"]["name"] == "my-uaa"):
        MY_UAA_URL  = line["metadata"]["url"]
        MY_UAA_GUID = line["metadata"]["guid"]
        break

print "MY_UAA_URL  " + MY_UAA_URL
print "MY_UAA_GUID " + MY_UAA_GUID
root = json.loads(check_output(["cf", "curl", MY_UAA_URL]))
#print root                 
print json.dumps(root, sort_keys=True, indent=4, separators=(',', ': '))

print "done"


Create UAA Service
secret code:········
Creating service instance my-uaa in org cwinsor@gmail.com / space dev as cwinsor@gmail.com...
OK

MY_UAA_URL /v2/service_instances/01b0f4be-5186-4173-84d4-19c11ef47834
MY_UAA_GUID01b0f4be-5186-4173-84d4-19c11ef47834
{
    "entity": {
        "credentials": {},
        "dashboard_url": null,
        "gateway_data": null,
        "last_operation": {
            "created_at": "2016-12-02T00:20:22Z",
            "description": "",
            "state": "succeeded",
            "type": "create",
            "updated_at": null
        },
        "name": "my-uaa",
        "routes_url": "/v2/service_instances/01b0f4be-5186-4173-84d4-19c11ef47834/routes",
        "service_bindings_url": "/v2/service_instances/01b0f4be-5186-4173-84d4-19c11ef47834/service_bindings",
        "service_keys_url": "/v2/service_instances/01b0f4be-5186-4173-84d4-19c11ef47834/service_keys",
        "service_plan_guid": "7b3e7f66-b1b8-448e-857e-79e2fde2e6c8",
        "service_plan_u

In [70]:
'''
cf env cwinsor.temp.html
produces
 "issuerId": "https://5b5428fa-4922-4be6-81ed-ad40a2a28f70.predix-uaa.run.aws-usw02-pr.ice.predix.io/oauth/token",

we want to find that using "cf curl"
cf curl /v2/spaces
produces
["resources"][0]["metadata"]["url"] = "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc",
["resources"][0]["entity"]["name"] = "dev"
["resources"][0]["entity"]["apps_url"] = "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/apps",
["resources"][0]["entity"]["service_instances_url"] = "/v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/service_instances",

<first find the UAA service GUID>
cf curl /v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/service_instances
produces
["resources"][0]["entity"]["name"]: "my-uaa"
["resources"][0]["metadata"]["guid"]: "5b5428fa-4922-4be6-81ed-ad40a2a28f70",

<now that we have the UAA GUID - search for that in my app as a service...>
cf curl /v2/spaces/f9c7e95d-76ad-477e-aaa8-ce8103b4e7cc/apps
produces
["resources"][0]["entity"]["name"]: "cwinsor.temp.html"
["resources"][0]["metadata"]["url"]: "/v2/apps/8508f927-aeb4-4f43-a62e-4348b13cb58d",
["resources"][0]["entity"]["service_bindings_url"]: "/v2/apps/8508f927-aeb4-4f43-a62e-4348b13cb58d/service_bindings",

cf curl /v2/apps/8508f927-aeb4-4f43-a62e-4348b13cb58d/service_bindings
produces
["resources"][0]["entity"][service_instance_guid": "5b5428fa-4922-4be6-81ed-ad40a2a28f70",
["resources"][0]["entity"]["credentials"]["issuerId"]: "https://5b5428fa-4922-4be6-81ed-ad40a2a28f70.predix-uaa.run.aws-usw02-pr.ice.predix.io/oauth/token",

GOT IT !
'''
x = 1
        

In [87]:
### Create application bound to UAA service (to get parameters around the UAA service)

print "create temporary application bound to UAA service"
foo = check_output(["echo", "", ">", "cwinsor.temp.html"])
foo = check_output(["cf", "push", "--no-start", "cwinsor.temp.html"])
foo = check_output(["cf", "bs", "cwinsor.temp.html", "my-uaa"])
foo = check_output(["cf", "env", "cwinsor.temp.html"])

# search for service bindings URL
print "search for service bindings URL"
root = json.loads(check_output(["cf", "curl", MY_PR_APPS_URL]))
#print json.dumps(root, sort_keys=True, indent=4, separators=(',', ': '))
for num, line in enumerate(root["resources"]):
    if (line["entity"]["name"] == "cwinsor.temp.html"):
        MY_APP_SERVICE_BINDINGS_URL = line["entity"]["service_bindings_url"]
        break

# search in the service bindings for issuerID where the service_instance_guid is the UUA
root = json.loads(check_output(["cf", "curl", MY_APP_SERVICE_BINDINGS_URL]))
#print json.dumps(root, sort_keys=True, indent=4, separators=(',', ': '))
for num, line in enumerate(root["resources"]):
    if (line["entity"]["service_instance_guid"] == MY_UAA_GUID):
        MY_APP_SERVICE_BINDINGS_URL = line["entity"]["credentials"]["issuerId"]
        break

print "MY_APP_SERVICE_BINDINGS_URL " + MY_APP_SERVICE_BINDINGS_URL

foo = check_output(["cf", "delete", "-f", "cwinsor.temp.html"])

print "done"

create temporary application bound to UAA service
search for service bindings URL
MY_APP_SERVICE_BINDINGS_URL https://01b0f4be-5186-4173-84d4-19c11ef47834.predix-uaa.run.aws-usw02-pr.ice.predix.io/oauth/token
done


In [ ]:
# UAAC Targets UAA
# The URL in the UAAC target command is the base URL of the UAA
# 15> uaac target https://2edfb28e-e02c-4ae9-a7cd-818c263128d8.predix-uaa.run.aws-usw02-pr.ice.predix.io
# 16> uaac token client get admin --secret my-secret # gets the admin token, NOT usable for asset


print "UAAC Targets UAA"
my_secret = getpass.getpass('secret code:')
authorities = "openid,uaa.none,uaa.resource," + "predix-asset.zones." + MY_APP_ASSET_INSTANCE_ID + ".user"
scope = "uaa.none,openid," + "predix-asset.zones." + MY_APP_ASSET_INSTANCE_ID + ".user"

#authorities = ["openid","uaa.none","uaa.resource"]
#scope = ["uaa.none","openid"]
       
foo = check_output(["uaac",
                    "target",
                     MY_APP_SERVICE_BINDINGS_URL,
                     MY_APP_SERVICE_BINDINGS_URL
                    
MY_UAA_URL /v2/service_instances/01b0f4be-5186-4173-84d4-19c11ef47834
                    
                    "add", "asset-client",
                    "--authorities", authorities,
                    "--scope", scope,
                    "--autoapprove", "openid",
                    "--authorized_grant_types", "authorization_code,client_credentials,refresh_token,password",
                    "--secret", my_secret,
                    "--name", "asset-client"])





In [72]:
### Create Asset Service
### cf create-service predix-asset Tiered my-asset -c '{"trustedIssuerIds":["https://2edfb28e-e02c-4ae9-a7cd-818c263128d8.predix-uaa.run.aws-usw02-pr.ice.predix.io/oauth/token"]}'

print("Create Asset Service")
trusted_issuer = json.dumps({"trustedIssuerIds": [MY_APP_SERVICE_BINDINGS_URL]})
#print trusted_issuer
foo = check_output(["cf", "create-service", "predix-asset", "Tiered", "my-asset", "-c", trusted_issuer])
#print foo

# now get the URL for the Asset
root = json.loads(check_output(["cf", "curl", MY_PR_SERVICES_URL]))
#print json.dumps(root, sort_keys=True, indent=4, separators=(',', ': '))
# look for "my-asset" service
for num, line in enumerate(root["resources"]):
    if (line["entity"]["name"] == "my-asset"):
        MY_ASSET_URL  = line["metadata"]["url"]
        MY_ASSET_GUID = line["metadata"]["guid"]
        break

print "MY_ASSET_URL " + MY_UAA_URL
print "MY_ASSET_GUID " + MY_UAA_GUID
root = json.loads(check_output(["cf", "curl", MY_ASSET_URL]))
#print root                 
print json.dumps(root, sort_keys=True, indent=4, separators=(',', ': '))

print "done"

Create Asset Service
MY_ASSET_URL /v2/service_instances/01b0f4be-5186-4173-84d4-19c11ef47834
MY_ASSET_GUID 01b0f4be-5186-4173-84d4-19c11ef47834
{
    "entity": {
        "credentials": {},
        "dashboard_url": null,
        "gateway_data": null,
        "last_operation": {
            "created_at": "2016-12-02T00:21:20Z",
            "description": "",
            "state": "succeeded",
            "type": "create",
            "updated_at": null
        },
        "name": "my-asset",
        "routes_url": "/v2/service_instances/998c3c4a-be3c-4e34-b8be-c71dc497554b/routes",
        "service_bindings_url": "/v2/service_instances/998c3c4a-be3c-4e34-b8be-c71dc497554b/service_bindings",
        "service_keys_url": "/v2/service_instances/998c3c4a-be3c-4e34-b8be-c71dc497554b/service_keys",
        "service_plan_guid": "e08d80b5-6da3-4f29-a2f1-f7834d06cd29",
        "service_plan_url": "/v2/service_plans/e08d80b5-6da3-4f29-a2f1-f7834d06cd29",
        "space_guid": "f9c7e95d-76ad-477e-aaa8-

In [73]:
### Create application bound to Asset service (to get parameters around the Asset service)

print "create temporary application bound to Asset service"
foo = check_output(["echo", "", ">", "cwinsor.temp.html"])
foo = check_output(["cf", "push", "--no-start", "cwinsor.temp.html"])
foo = check_output(["cf", "bs", "cwinsor.temp.html", "my-asset"])
foo = check_output(["cf", "env", "cwinsor.temp.html"])

# search for service bindings URL
print "search for service bindings URL"
root = json.loads(check_output(["cf", "curl", MY_PR_APPS_URL]))
print json.dumps(root, sort_keys=True, indent=4, separators=(',', ': '))
for num, line in enumerate(root["resources"]):
    if (line["entity"]["name"] == "cwinsor.temp.html"):
        MY_APP_SERVICE_BINDINGS_URL = line["entity"]["service_bindings_url"]
        break

print "MY_APP_SERVICE_BINDINGS_URL " + MY_APP_SERVICE_BINDINGS_URL

# search in the service bindings for service instance with Asset GUID
root = json.loads(check_output(["cf", "curl", MY_APP_SERVICE_BINDINGS_URL]))
print json.dumps(root, sort_keys=True, indent=4, separators=(',', ': '))
for num, line in enumerate(root["resources"]):
    if (line["entity"]["service_instance_guid"] == MY_ASSET_GUID):
        MY_APP_ASSET_INSTANCE_ID = line["entity"]["credentials"]["instanceId"]
        break

print "MY_APP_ASSET_INSTANCE_ID " + MY_APP_ASSET_INSTANCE_ID

print "done"

create temporary application bound to Asset service
search for service bindings URL
{
    "next_url": null,
    "prev_url": null,
    "resources": [
        {
            "entity": {
                "buildpack": null,
                "command": null,
                "console": false,
                "debug": null,
                "detected_buildpack": null,
                "detected_buildpack_guid": null,
                "detected_start_command": "",
                "diego": false,
                "disk_quota": 1024,
                "docker_credentials_json": {
                    "redacted_message": "[PRIVATE DATA HIDDEN]"
                },
                "docker_image": null,
                "enable_ssh": true,
                "environment_json": {},
                "events_url": "/v2/apps/b3584d25-f737-404e-8a9e-e765f5bc89ba/events",
                "health_check_timeout": null,
                "health_check_type": "port",
                "instances": 1,
                "memory": 

In [104]:
# UAAC Targets UAA
# The URL in the UAAC target command is the base URL of the UAA. 
# 15> uaac target https://2edfb28e-e02c-4ae9-a7cd-818c263128d8.predix-uaa.run.aws-usw02-pr.ice.predix.io
# 16> uaac token client get admin --secret my-secret # gets the admin token, NOT usable for asset

#print MY_APP_SERVICE_BINDINGS_URL

print "UAAC Targets UAA"

bindings_url = MY_APP_SERVICE_BINDINGS_URL.split("/oauth/token")
foo = check_output(["uaac", "target", bindings_url[0]])

my_secret = getpass.getpass('secret code:')
print my_secret
# 16> uaac token client get admin --secret my-secret # gets the admin token, NOT usable for asset
#foo = check_output("uaac","token","client","get","admin","--secret", my_secret)
foo = check_output("uaac","token","client","get","admin", "--secret", "xxxxx")

print foo



#MY_APP_SERVICE_BINDINGS_URL /v2/apps/5722127e-1f0e-4044-a2ed-d0122c7a9db3/service_bindings
#MY_APP_SERVICE_BINDINGS_URL https://01b0f4be-5186-4173-84d4-19c11ef47834.predix-uaa.run.aws-usw02-pr.ice.predix.io/oauth/token
print "done"




UAAC Targets UAA
secret code:········
xxxxx


TypeError: __init__() got multiple values for keyword argument 'stdout'

In [105]:
from subprocess import Popen, PIPE
from subprocess import check_output


fw = open("tmpout", "wb")
fr = open("tmpout", "r")
p = Popen("./a.out", stdin = PIPE, stdout = fw, stderr = fw, bufsize = 1)
p.stdin.write("1\n")
out = fr.read()
p.stdin.write("5\n")
out = fr.read()
fw.close()
fr.close()


OSError: [Errno 2] No such file or directory

In [95]:
 

src = "ice cream"
blah = src.split("ea")
print blah


['ice cr', 'm']


In [85]:
# Add UAAC Client
'''
uaac client add asset-client 
--authorities openid,uaa.none,uaa.resource,predix-asset.zones.6827f028-5b6b-439e-ae08-9587cedb6f56.user
--scope uaa.none,openid,predix-asset.zones.6827f028-5b6b-439e-ae08-9587cedb6f56.user
--autoapprove openid
--authorized_grant_types authorization_code,client_credentials,refresh_token,password
--secret my-secret
--name asset-client
'''

print "Add UAAC Client"
my_secret = getpass.getpass('secret code:')
authorities = "openid,uaa.none,uaa.resource," + "predix-asset.zones." + MY_APP_ASSET_INSTANCE_ID + ".user"
scope = "uaa.none,openid," + "predix-asset.zones." + MY_APP_ASSET_INSTANCE_ID + ".user"

#authorities = ["openid","uaa.none","uaa.resource"]
#scope = ["uaa.none","openid"]
       
foo = check_output(["uaac", "client",
                    "add", "asset-client",
                    "--authorities", authorities,
                    "--scope", scope,
                    "--autoapprove", "openid",
                    "--authorized_grant_types", "authorization_code,client_credentials,refresh_token,password",
                    "--secret", my_secret,
                    "--name", "asset-client"])

print foo
#want
#str = '
#uaac client
#add asset-client
#--authorities openid,uaa.none,uaa.resource,predix-asset.zones.6827f028-5b6b-439e-ae08-9587cedb6f56.user
#--scope uaa.none,openid,predix-asset.zones.6827f028-5b6b-439e-ae08-9587cedb6f56.user
#--autoapprove openid
#--authorized_grant_types authorization_code,client_credentials,refresh_token,password
#--secret my-secret
#--name asset-client'

#'uaac client add asset-client --authorities openid,uaa.none,uaa.resource,predix-asset.zones.6827f028-5b6b-439e-ae08-9587cedb6f56.user --scope uaa.none,openid,predix-asset.zones.6827f028-5b6b-439e-ae08-9587cedb6f56.user --autoapprove openid --authorized_grant_types authorization_code,client_credentials,refresh_token,password --secret my-secret --name asset-client
#'uaac client add asset-client --authorities openid,uaa.none,uaa.resource,predix-asset.zones.998c3c4a-be3c-4e34-b8be-c71dc497554b.user --scope uaa.none,openid,predix-asset.zones.998c3c4a-be3c-4e34-b8be-c71dc497554b.user --autoapprove openid --authorized_grant_types authorization_code,client_credentials,refresh_token,password --name asset-client

#998c3c4a-be3c-4e34-b8be-c71dc497554b.



Add UAAC Client
secret code:········


CalledProcessError: Command '['uaac', 'client', 'add', 'asset-client', '--authorities', u'openid,uaa.none,uaa.resource,predix-asset.zones.998c3c4a-be3c-4e34-b8be-c71dc497554b.user', '--scope', u'uaa.none,openid,predix-asset.zones.998c3c4a-be3c-4e34-b8be-c71dc497554b.user', '--autoapprove', 'openid', '--authorized_grant_types', 'authorization_code,client_credentials,refresh_token,password', '--secret', 'xxxxx', '--name', 'asset-client']' returned non-zero exit status 1

In [76]:
foo = check_output(["uaac", "client", "--version"])
print foo


UAA client 3.4.0



In [84]:

foo = authorities = ["openid","uaa.none","uaa.resource"," + "predix-asset.zones." + MY_APP_ASSET_INSTANCE_ID + ".user"
print foo


openid,uaa.none,uaa.resource,predix-asset.zones.998c3c4a-be3c-4e34-b8be-c71dc497554b.user
